In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_date,current_timestamp
from pyspark.sql.functions import *

In [3]:
spark=SparkSession.builder.\
config('spark.shuffle.useOldFetchProtocol', 'true').\
enableHiveSupport().\
appName("Project_Pyspark").\
master('yarn').\
getOrCreate()

In [4]:
spark

In [54]:
loans_raw_df=spark.read.format("csv").option("inferSchema",True)\
.option("header",True).load("/user/itv012010/lendingclubproject/raw/loans_data_csv")

In [55]:
loans_raw_df.show() #to check data 

+-------+--------------------+---------+-----------+---------+--------+-----------+--------+-----------+--------------------+--------------------+
|loan_id|           member_id|loan_amnt|funded_amnt|     term|int_rate|installment| issue_d|loan_status|             purpose|               title|
+-------+--------------------+---------+-----------+---------+--------+-----------+--------+-----------+--------------------+--------------------+
| 491699|961ae110ec063761f...|   7000.0|     7000.0|36 months|   12.73|     234.97|Mar-2010| Fully Paid|  debt_consolidation|Noelle's debt con...|
| 491685|291854d244748c911...|  15000.0|    15000.0|36 months|   10.62|      488.4|Mar-2010| Fully Paid|               other|           Education|
| 491667|b5cf66b90193da9d2...|   6400.0|     6400.0|36 months|    7.88|      200.2|Mar-2010| Fully Paid|  debt_consolidation|   Make money easily|
| 491160|7f53fd1acb7759e5b...|   4000.0|     4000.0|36 months|   14.59|     137.86|Mar-2010| Fully Paid|             w

In [7]:
loans_raw_df.printSchema() #we can see data types and renaming of col is required

root
 |-- loan_id: string (nullable = true)
 |-- member_id: string (nullable = true)
 |-- loan_amnt: double (nullable = true)
 |-- funded_amnt: double (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: double (nullable = true)
 |-- issue_d: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- title: string (nullable = true)



In [56]:
loans_schema = "loan_id string,member_id string,loan_amount float,funded_amount float,\
                loan_term_months string,interest_rate float,monthly_installment float,issue_date string,\
                loan_status string,loan_purpose string,loan_title string"
#we are renaming few col's along with data types of the same.

In [57]:
loans_raw_df=spark.read.format("csv").schema(loans_schema) \
.option("header",True).load("/user/itv012010/lendingclubproject/raw/loans_data_csv")

In [58]:
loans_raw_df.printSchema() #we can see meta data is updated

root
 |-- loan_id: string (nullable = true)
 |-- member_id: string (nullable = true)
 |-- loan_amount: float (nullable = true)
 |-- funded_amount: float (nullable = true)
 |-- loan_term_months: string (nullable = true)
 |-- interest_rate: float (nullable = true)
 |-- monthly_installment: float (nullable = true)
 |-- issue_date: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- loan_purpose: string (nullable = true)
 |-- loan_title: string (nullable = true)



In [59]:
loans_df_ingested=loans_raw_df.withColumn("ingest_date",current_timestamp())

In [60]:
loans_df_ingested.select("ingest_date").show() #we can see data is present

+--------------------+
|         ingest_date|
+--------------------+
|2024-03-30 12:49:...|
|2024-03-30 12:49:...|
|2024-03-30 12:49:...|
|2024-03-30 12:49:...|
|2024-03-30 12:49:...|
|2024-03-30 12:49:...|
|2024-03-30 12:49:...|
|2024-03-30 12:49:...|
|2024-03-30 12:49:...|
|2024-03-30 12:49:...|
|2024-03-30 12:49:...|
|2024-03-30 12:49:...|
|2024-03-30 12:49:...|
|2024-03-30 12:49:...|
|2024-03-30 12:49:...|
|2024-03-30 12:49:...|
|2024-03-30 12:49:...|
|2024-03-30 12:49:...|
|2024-03-30 12:49:...|
|2024-03-30 12:49:...|
+--------------------+
only showing top 20 rows



In [61]:
loans_df_ingested.createOrReplaceTempView("loans") 

In [62]:
loans_df_ingested

loan_id,member_id,loan_amount,funded_amount,loan_term_months,interest_rate,monthly_installment,issue_date,loan_status,loan_purpose,loan_title,ingest_date
491699,961ae110ec063761f...,7000.0,7000.0,36 months,12.73,234.97,Mar-2010,Fully Paid,debt_consolidation,Noelle's debt con...,2024-03-30 12:49:...
491685,291854d244748c911...,15000.0,15000.0,36 months,10.62,488.4,Mar-2010,Fully Paid,other,Education,2024-03-30 12:49:...
491667,b5cf66b90193da9d2...,6400.0,6400.0,36 months,7.88,200.2,Mar-2010,Fully Paid,debt_consolidation,Make money easily,2024-03-30 12:49:...
491160,7f53fd1acb7759e5b...,4000.0,4000.0,36 months,14.59,137.86,Mar-2010,Fully Paid,wedding,wedding expenses,2024-03-30 12:49:...
491675,12edc8e328b993914...,20000.0,20000.0,36 months,13.85,682.08,Mar-2010,Fully Paid,other,Short Term Tax Loan,2024-03-30 12:49:...
491668,e18a847f091332ff6...,6000.0,6000.0,36 months,11.36,197.47,Mar-2010,Charged Off,debt_consolidation,GREAT BORROWER --...,2024-03-30 12:49:...
491663,d6fe0000e876ab76d...,5500.0,5500.0,36 months,11.36,181.02,Mar-2010,Fully Paid,credit_card,Road to Success,2024-03-30 12:49:...
491632,62c38934f3a2ebcd2...,10000.0,10000.0,36 months,15.7,350.11,Mar-2010,Fully Paid,credit_card,Finish Paying off...,2024-03-30 12:49:...
491618,7701762ef5d302b0d...,25000.0,25000.0,36 months,15.33,870.71,Mar-2010,Fully Paid,debt_consolidation,Pay off my high i...,2024-03-30 12:49:...
491622,42a2df45b02b21b9a...,25000.0,25000.0,36 months,16.07,879.85,Mar-2010,Fully Paid,debt_consolidation,Debt Pay Off,2024-03-30 12:49:...


In [63]:
spark.sql("select count(*) from loans") #total records in loans data 

count(1)
2260701


In [64]:
spark.sql("select count(*) from loans where loan_amount is null") #to check nulls in loan_Amount there are 33 records where nulls are present

count(1)
33


In [65]:
spark.sql("select * from loans where loan_amount is null") #we can see there are many nulls present

loan_id,member_id,loan_amount,funded_amount,loan_term_months,interest_rate,monthly_installment,issue_date,loan_status,loan_purpose,loan_title,ingest_date
Loans that do not...,e3b0c44298fc1c149...,null,null,null,null,null,null,null,null,null,2024-03-30 12:49:...
Total amount fund...,e3b0c44298fc1c149...,null,null,null,null,null,null,null,null,null,2024-03-30 12:49:...
Total amount fund...,e3b0c44298fc1c149...,null,null,null,null,null,null,null,null,null,2024-03-30 12:49:...
Total amount fund...,e3b0c44298fc1c149...,null,null,null,null,null,null,null,null,null,2024-03-30 12:49:...
Total amount fund...,e3b0c44298fc1c149...,null,null,null,null,null,null,null,null,null,2024-03-30 12:49:...
Total amount fund...,e3b0c44298fc1c149...,null,null,null,null,null,null,null,null,null,2024-03-30 12:49:...
Total amount fund...,e3b0c44298fc1c149...,null,null,null,null,null,null,null,null,null,2024-03-30 12:49:...
Total amount fund...,e3b0c44298fc1c149...,null,null,null,null,null,null,null,null,null,2024-03-30 12:49:...
Total amount fund...,e3b0c44298fc1c149...,null,null,null,null,null,null,null,null,null,2024-03-30 12:49:...
Total amount fund...,e3b0c44298fc1c149...,null,null,null,null,null,null,null,null,null,2024-03-30 12:49:...


In [66]:
columns_to_check=["loan_amount","funded_amount","loan_term_months","interest_rate","monthly_installment","issue_date",\
                 "loan_status","loan_purpose"]

In [67]:
loans_filtered_df=loans_df_ingested.na.drop(subset=columns_to_check) #if any of these cols are null it will drop 

In [68]:
loans_filtered_df.count() #count after dropping nulls

2260667

In [69]:
loans_filtered_df.createOrReplaceTempView("loans")

In [70]:
loans_filtered_df #we have to change loan_term_months to years 

loan_id,member_id,loan_amount,funded_amount,loan_term_months,interest_rate,monthly_installment,issue_date,loan_status,loan_purpose,loan_title,ingest_date
491699,961ae110ec063761f...,7000.0,7000.0,36 months,12.73,234.97,Mar-2010,Fully Paid,debt_consolidation,Noelle's debt con...,2024-03-30 12:50:...
491685,291854d244748c911...,15000.0,15000.0,36 months,10.62,488.4,Mar-2010,Fully Paid,other,Education,2024-03-30 12:50:...
491667,b5cf66b90193da9d2...,6400.0,6400.0,36 months,7.88,200.2,Mar-2010,Fully Paid,debt_consolidation,Make money easily,2024-03-30 12:50:...
491160,7f53fd1acb7759e5b...,4000.0,4000.0,36 months,14.59,137.86,Mar-2010,Fully Paid,wedding,wedding expenses,2024-03-30 12:50:...
491675,12edc8e328b993914...,20000.0,20000.0,36 months,13.85,682.08,Mar-2010,Fully Paid,other,Short Term Tax Loan,2024-03-30 12:50:...
491668,e18a847f091332ff6...,6000.0,6000.0,36 months,11.36,197.47,Mar-2010,Charged Off,debt_consolidation,GREAT BORROWER --...,2024-03-30 12:50:...
491663,d6fe0000e876ab76d...,5500.0,5500.0,36 months,11.36,181.02,Mar-2010,Fully Paid,credit_card,Road to Success,2024-03-30 12:50:...
491632,62c38934f3a2ebcd2...,10000.0,10000.0,36 months,15.7,350.11,Mar-2010,Fully Paid,credit_card,Finish Paying off...,2024-03-30 12:50:...
491618,7701762ef5d302b0d...,25000.0,25000.0,36 months,15.33,870.71,Mar-2010,Fully Paid,debt_consolidation,Pay off my high i...,2024-03-30 12:50:...
491622,42a2df45b02b21b9a...,25000.0,25000.0,36 months,16.07,879.85,Mar-2010,Fully Paid,debt_consolidation,Debt Pay Off,2024-03-30 12:50:...


In [71]:
loans_term_modified_df=loans_filtered_df.withColumn(
"loan_term_months",
    (regexp_replace(col("loan_term_months"),"months","").cast("int")/12)\
    .cast("int")).withColumnRenamed("loan_term_months","loan_term_years")

In [72]:
loans_term_modified_df #we can see loan_term_years is changed 

loan_id,member_id,loan_amount,funded_amount,loan_term_years,interest_rate,monthly_installment,issue_date,loan_status,loan_purpose,loan_title,ingest_date
491699,961ae110ec063761f...,7000.0,7000.0,3,12.73,234.97,Mar-2010,Fully Paid,debt_consolidation,Noelle's debt con...,2024-03-30 12:50:...
491685,291854d244748c911...,15000.0,15000.0,3,10.62,488.4,Mar-2010,Fully Paid,other,Education,2024-03-30 12:50:...
491667,b5cf66b90193da9d2...,6400.0,6400.0,3,7.88,200.2,Mar-2010,Fully Paid,debt_consolidation,Make money easily,2024-03-30 12:50:...
491160,7f53fd1acb7759e5b...,4000.0,4000.0,3,14.59,137.86,Mar-2010,Fully Paid,wedding,wedding expenses,2024-03-30 12:50:...
491675,12edc8e328b993914...,20000.0,20000.0,3,13.85,682.08,Mar-2010,Fully Paid,other,Short Term Tax Loan,2024-03-30 12:50:...
491668,e18a847f091332ff6...,6000.0,6000.0,3,11.36,197.47,Mar-2010,Charged Off,debt_consolidation,GREAT BORROWER --...,2024-03-30 12:50:...
491663,d6fe0000e876ab76d...,5500.0,5500.0,3,11.36,181.02,Mar-2010,Fully Paid,credit_card,Road to Success,2024-03-30 12:50:...
491632,62c38934f3a2ebcd2...,10000.0,10000.0,3,15.7,350.11,Mar-2010,Fully Paid,credit_card,Finish Paying off...,2024-03-30 12:50:...
491618,7701762ef5d302b0d...,25000.0,25000.0,3,15.33,870.71,Mar-2010,Fully Paid,debt_consolidation,Pay off my high i...,2024-03-30 12:50:...
491622,42a2df45b02b21b9a...,25000.0,25000.0,3,16.07,879.85,Mar-2010,Fully Paid,debt_consolidation,Debt Pay Off,2024-03-30 12:50:...


In [73]:
loans_term_modified_df.createOrReplaceTempView("loans")

In [36]:
spark.sql("""select loan_purpose,count(*) as total from loans
             group by loan_purpose 
             order by total desc          
""") # we need to add loan_purpose in others apart from regular ones 

loan_purpose,total
debt_consolidation,1277790
credit_card,516926
home_improvement,150440
other,139413
major_purchase,50429
medical,27481
small_business,24659
car,24009
vacation,15525
moving,15402


In [74]:
loan_purpose_lookup = ["debt_consolidation", "credit_card",
"home_improvement", "other", "major_purchase", "medical", "small_business",
"car", "vacation", "moving", "house", "wedding", "renewable_energy",
"educational"] 

In [75]:
loans_purpose_modified=loans_term_modified_df.withColumn("loan_purpose",
                                 when (col("loan_purpose").isin(loan_purpose_lookup),col("loan_purpose")).otherwise("other"))

In [76]:
loans_purpose_modified.createOrReplaceTempView("loans")

In [77]:
spark.sql("""select loan_purpose,count(*) as total from loans
             group by loan_purpose 
             order by total desc          
""") #now we can see loan_purpose in others apart from regular ones 

loan_purpose,total
debt_consolidation,1277790
credit_card,516926
home_improvement,150440
other,139667
major_purchase,50429
medical,27481
small_business,24659
car,24009
vacation,15525
moving,15402


In [78]:
loans_purpose_modified\
.groupBy("loan_purpose")\
.agg(count('*').alias("total"))\
.orderBy(col("total").desc())  #this is same as above query using dataframes

loan_purpose,total
debt_consolidation,1277790
credit_card,516926
home_improvement,150440
other,139667
major_purchase,50429
medical,27481
small_business,24659
car,24009
vacation,15525
moving,15402


In [79]:
loans_purpose_modified #cleaned data 

loan_id,member_id,loan_amount,funded_amount,loan_term_years,interest_rate,monthly_installment,issue_date,loan_status,loan_purpose,loan_title,ingest_date
491699,961ae110ec063761f...,7000.0,7000.0,3,12.73,234.97,Mar-2010,Fully Paid,debt_consolidation,Noelle's debt con...,2024-03-30 12:51:...
491685,291854d244748c911...,15000.0,15000.0,3,10.62,488.4,Mar-2010,Fully Paid,other,Education,2024-03-30 12:51:...
491667,b5cf66b90193da9d2...,6400.0,6400.0,3,7.88,200.2,Mar-2010,Fully Paid,debt_consolidation,Make money easily,2024-03-30 12:51:...
491160,7f53fd1acb7759e5b...,4000.0,4000.0,3,14.59,137.86,Mar-2010,Fully Paid,wedding,wedding expenses,2024-03-30 12:51:...
491675,12edc8e328b993914...,20000.0,20000.0,3,13.85,682.08,Mar-2010,Fully Paid,other,Short Term Tax Loan,2024-03-30 12:51:...
491668,e18a847f091332ff6...,6000.0,6000.0,3,11.36,197.47,Mar-2010,Charged Off,debt_consolidation,GREAT BORROWER --...,2024-03-30 12:51:...
491663,d6fe0000e876ab76d...,5500.0,5500.0,3,11.36,181.02,Mar-2010,Fully Paid,credit_card,Road to Success,2024-03-30 12:51:...
491632,62c38934f3a2ebcd2...,10000.0,10000.0,3,15.7,350.11,Mar-2010,Fully Paid,credit_card,Finish Paying off...,2024-03-30 12:51:...
491618,7701762ef5d302b0d...,25000.0,25000.0,3,15.33,870.71,Mar-2010,Fully Paid,debt_consolidation,Pay off my high i...,2024-03-30 12:51:...
491622,42a2df45b02b21b9a...,25000.0,25000.0,3,16.07,879.85,Mar-2010,Fully Paid,debt_consolidation,Debt Pay Off,2024-03-30 12:51:...


In [80]:
loans_purpose_modified.write\
.format("csv")\
.mode("overwrite")\
.option("path","/user/itv012010/lendingclubproject/cleaned/loans_csv")\
.save() #Finally saving this cleaned data in csv format

In [81]:
loans_purpose_modified.write\
.format("parquet")\
.mode("overwrite")\
.option("path","/user/itv012010/lendingclubproject/cleaned/loans_parquet")\
.save() #Finally saving this cleaned data in parquet format